In [1]:
import os
import requests
from dotenv import load_dotenv
from mistralai import Mistral

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave de API de Mistral desde las variables de entorno
api_key = os.getenv('MISTRAL_API_KEY')
client = Mistral(api_key=api_key)
model = "mistral-small-latest"

In [2]:
system_prompt = f"""
Your job is to help create interesting fantasy worlds that \
players would love to play in.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery.
- You must stay below 3-5 sentences for each description.
"""

world_prompt = f"""
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:"""


output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_prompt}
    ],
)

In [3]:
world_output =output.choices[0].message.content
print(world_output)
world_output = world_output.strip()
world = {
    "name": world_output.split('\n')[0].strip()
    .replace('World Name: ', ''),
    "description": '\n'.join(world_output.split('\n')[1:])
    .replace('World Description:', '').strip()
}

World Name: The Skyborne Dominion

World Description: In the Skyborne Dominion, cities float among the clouds, perched atop the colossal backs of sentient, sky-beasts called Aetherstriders. These gentle giants migrate through the skies, carrying entire civilizations with them. Humans, elves, and other races live in harmony, adapting their architecture and culture to the ever-changing landscape of the heavens. Below, the surface world is a mysterious and dangerous place, rarely visited and shrouded in legend.


In [15]:
kingdom_prompt = f"""
Create 3 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in. \
Describe important leaders, cultures, history of the kingdom.\

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>
Kingdom 3 Name: <KINGDOM NAME>
Kingdom 3 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1"""


output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": kingdom_prompt}
    ],
)

In [17]:
kingdoms_output = output.choices[0].message.content
print(kingdoms_output)

Kingdom 1 Name: Aerium
Kingdom 1 Description: Aerium is a bustling kingdom known for its advanced airship technology and vibrant markets. Queen Lyria, a skilled navigator and diplomat, rules with a focus on innovation and trade. The Aerians are a diverse mix of races, united by their love for exploration and discovery. Their history is marked by daring expeditions into the unknown skies and alliances with other floating kingdoms.

Kingdom 2 Name: Lumynara
Kingdom 2 Description: Lumynara is a kingdom of light and knowledge, home to the wise and enigmatic Elven Council. The elves of Lumynara are renowned for their magical prowess and ancient libraries. Their history is one of preservation and guidance, often mediating conflicts between other kingdoms. The Lumynaran culture values wisdom, harmony, and the pursuit of magical understanding.

Kingdom 3 Name: Caelum
Kingdom 3 Description: Caelum is a kingdom of warriors and craftsmen, led by the stern but fair Marshal Thorne. The Caelumites a

In [18]:
kingdoms = {}
kingdoms_output = output.choices[0].message.content

for output in kingdoms_output.split('\n\n'):
  kingdom_name = output.strip().split('\n')[0] \
    .split('Name: ')[1].strip()
  print(f'Created kingdom "{kingdom_name}" in {world["name"]}')
  kingdom_description = output.strip().split('\n')[1] \
    .split('Description: ')[1].strip()
  kingdom = {
      "name": kingdom_name,
      "description": kingdom_description,
      "world": world['name']
  }
  kingdoms[kingdom_name] = kingdom
world['kingdoms'] = kingdoms

print(f'\nKingdom 1 Description: \
{kingdom["description"]}')

Created kingdom "Aerium" in The Skyborne Dominion
Created kingdom "Lumynara" in The Skyborne Dominion
Created kingdom "Caelum" in The Skyborne Dominion

Kingdom 1 Description: Caelum is a kingdom of warriors and craftsmen, led by the stern but fair Marshal Thorne. The Caelumites are known for their exceptional blacksmithing and martial prowess. Their history is filled with tales of defense against sky-pirates and surface-dwelling monsters. The Caelumite culture emphasizes strength, honor, and the protection of their floating home.


In [ ]:
def get_town_prompt(world, kingdom):
    return f"""
    Create 3 different towns for a fantasy kingdom and world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \
    
    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}
    
    Town 1 Name:"""
    
import time

import time  # Asegúrate de importar la biblioteca time

def create_towns(world, kingdom):
    print(f'\nCreating towns for kingdom: {kingdom["name"]}...')
    output = client.chat.complete(
      model=model,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_town_prompt(world, kingdom)}
      ],
      temperature=0.9,
  )
    towns_output = output.choices[0].message.content
    
    towns = {}
    for output in towns_output.split('\nCreating towns for kingdom: {kingdom["name"]}...'):
        for num in range(1, 4):
            print(output.split(f"Town {num} Name: ")[1].split("\n")[0])
            print(output.split(f"Town {num} Description: ")[1].split("\n")[0])
            print("\n")
         
            town_name = output.split(f"Town {num} Name: ")[1].split("\n")[0] 
            town_description = output.split(f"Town {num} Description: ")[1].split("\n")[0]
        
            town = {
              "name": town_name,
              "description": town_description,
              "world": world['name'],
              "kingdom": kingdom['name']
            }
            towns[town_name] = town
        kingdom["towns"] = towns

In [84]:
for kingdom in kingdoms.values():
    create_towns(world, kingdom)  

first_kingdom = list(kingdoms.values())[0]
print(first_kingdom)
town = list(first_kingdom['towns'].values())[0]
print(f'\nTown 1 Description: {town["description"]}')


Creating towns for kingdom: Aerium...
Stellara
Nestled in the heart of the Whispering Clouds region, Stellara is a town known for its luminescent flora and the annual Starlight Festival. The town's most important place is the Great Observatory, a towering structure where astronomers study the night sky. Legend has it that Stellara was founded by a group of stargazers who followed a falling star to its landing site, where they discovered a hidden treasure trove of ancient knowledge.


Zephyr's Hollow
Located in the Windward Peaks, Zephyr's Hollow is a town carved into the cliffs, with homes and shops built into the rock faces. The town's most important place is the Wind Temple, dedicated to the spirit of the winds. The town has a rich history of wind-harnessing technology, and its inhabitants are skilled in creating wind-powered devices. It is said that the town was once a secret hideout for a legendary band of sky-pirates.


Mistveil
Situated in the Foggy Abyss, Mistveil is a town shr

In [85]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \
    
    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}
    
    Town Name: {town['name']}
    Town Description: {town['description']}
    
    Character 1 Name:"""
    
def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    output = client.chat.complete(
      model= model,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_npc_prompt(world, kingdom, town)}
      ],
        temperature=0.7 
    )

    npcs_output = output.choices[0].message.content
    npcs = {}
    for output in npcs_output.split('\n\n'):
        npc_name = output.strip().split('\n')[0]\
        .split('Name: ')[1].strip()
        print(f'- "{npc_name}" created')
        
        npc_description = output.strip().split('\n')[1\
        ].split('Description: ')[1].strip()
        
        npc = {
        "name": npc_name,
        "description": npc_description,
        "world": world['name'],
        "kingdom": kingdom['name'],
        "town": town['name']
        }
        npcs[npc_name] = npc
    town["npcs"] = npcs
    
for kingdom in kingdoms.values():
    for town in kingdom['towns'].values():
        create_npcs(world, kingdom, town)
  # For now we'll only generate npcs for one kingdom
    break

npc = list(town['npcs'].values())[0]

print(f'\nNPC 1 in {town["name"]}, \
{kingdom["name"]}:\n{npc["description"]}')


Creating characters for the town of: Stellara...
- "Eamon Skyweaver" created


IndexError: list index out of range